<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/reports/hold_shelf_reports_supply_demand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHPL - Collection Analysis - **Hold-Shelves Supply & Demand**
<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="300"/>

This report provides some queries and generates some visualizations for data
related to items that have been delievered to various hold-shelf locations in
the CHPL system.

Data sources:

* https://ilsweb.cincinnatilibrary.org/collection-analysis/
* https://github.com/plch/plch-holds-shelf

In [1]:
import pandas as pd
import numpy as np
import altair as alt
from urllib.parse import urlencode

chpl_collection_url = 'https://ilsweb.cincinnatilibrary.org/collection-analysis/current_collection'

start_date = '2021-11'

In [2]:
# Hold-shelf Supply
# where are items coming from to fill holds?

"""
NOTE: this excludes:
  * items where the source branch is the same same as the pickup branch
  * items having item types where there are not more than 100 of that type in the set 
"""

# 1. Main Vs. Everyone
sql = """\
with hold_shelf_data as (
  select
    date(modified_epoch, 'unixepoch', 'localtime') as date_hold_on_holdshelf,
    date(placed_epoch, 'unixepoch', 'localtime') as date_hold_placed,
    -- cast(
    --  round((modified_epoch - placed_epoch) / 86400.0) as integer
    -- ) as days_to_holdshelf,
    s_location_code as item_source_location_code,
    (
      select
        coalesce(branch_name.name, s_location_code, '')
      from
        "location"
        left outer join branch on branch.code_num = "location".branch_code_num
        left outer join branch_name on branch_name.branch_id = branch.id
      where
        "location".code = s_location_code
      limit
        1
    ) as item_source_branch_name,
    item.item_format,
    pickup_location_code,
    (
      select
        coalesce(branch_name.name, pickup_location_code, '')
      from
        "location"
        left outer join branch on branch.code_num = "location".branch_code_num
        left outer join branch_name on branch_name.branch_id = branch.id
      where
        "location".code = pickup_location_code
      limit
        1
    ) as item_pickup_branch_name
  from
    holds_shelf
    left outer join item as item on item.item_record_num = holds_shelf.record_num
  where
    modified_epoch >= CAST(
      strftime('%s', :start_date || '-01') AS INT
    )
    and modified_epoch < CAST(
      strftime('%s', DATE(:start_date || '-01', '+1 months')) AS INT
    )
)
select
  :start_date as month,
  case
    when item_source_branch_name != 'Main Library' then "All Branches"
    else item_source_branch_name
  end as item_source_branch,
  -- item_source_branch_name,
  -- item_format,
  count(*) as "count all item types"
from
  hold_shelf_data
where
  item_source_branch_name != item_pickup_branch_name
group by
  1,
  2
"""

sql_2 = """\
with hold_shelf_data as (
  select
    date(modified_epoch, 'unixepoch', 'localtime') as date_hold_on_holdshelf,
    date(placed_epoch, 'unixepoch', 'localtime') as date_hold_placed,
    -- cast(
    --  round((modified_epoch - placed_epoch) / 86400.0) as integer
    -- ) as days_to_holdshelf,
    s_location_code as item_source_location_code,
    (
      select
        coalesce(branch_name.name, s_location_code, '')
      from
        "location"
        left outer join branch on branch.code_num = "location".branch_code_num
        left outer join branch_name on branch_name.branch_id = branch.id
      where
        "location".code = s_location_code
      limit
        1
    ) as item_source_branch_name,
    item.item_format,
    pickup_location_code,
    (
      select
        coalesce(branch_name.name, pickup_location_code, '')
      from
        "location"
        left outer join branch on branch.code_num = "location".branch_code_num
        left outer join branch_name on branch_name.branch_id = branch.id
      where
        "location".code = pickup_location_code
      limit
        1
    ) as item_pickup_branch_name
  from
    holds_shelf
    left outer join item as item on item.item_record_num = holds_shelf.record_num
  where
    modified_epoch >= CAST(
      strftime('%s', :start_date || '-01') AS INT
    )
    and modified_epoch < CAST(
      strftime('%s', DATE(:start_date || '-01', '+1 months')) AS INT
    )
)
select
  :start_date as month,
  case
    when item_source_branch_name != 'Main Library' then "All Branches"
    else item_source_branch_name
  end as item_source_branch,
  -- item_source_branch_name,
  item_format,
  count(*) as count
from
  hold_shelf_data
where
  item_source_branch_name != item_pickup_branch_name
group by
  1,
  2,
  3
"""

df = pd.read_csv(
        chpl_collection_url + '.csv?' + urlencode(query={'sql': sql, 'start_date': start_date})
)

df_2 = pd.read_csv(
        chpl_collection_url + '.csv?' + urlencode(query={'sql': sql_2, 'start_date': start_date})
)

alt.Chart(df[df['count all item types'] > 100]).mark_bar().encode(
    x=alt.X('count all item types:Q'),
    y=alt.Y('item_source_branch', sort='-x'),
    # color='item_format',
    tooltip=['month', 'count all item types', 'item_source_branch']
).properties(
    title = f"Hold-shelf Supply - All Branches Vs Main - {df.iloc[0]['month']}",
    width = 800
).display()

alt.Chart(df_2[df_2['count'] > 100]).mark_bar().encode(
    x=alt.X('count:Q'),
    y=alt.Y('item_source_branch', sort='-x'),
    color='item_format',
    tooltip=['month', 'item_format', 'count', 'item_source_branch']
).properties(
    title = f"Hold-shelf Supply - All Branches Vs Main - {df.iloc[0]['month']}",
    width = 800
).display()

alt.Chart(...)

alt.Chart(...)

In [3]:
"""
What does the number of items used to fill holds look like for: 
  * items that have the same source location as the pickup hold shelf
  * items that have an different source location compared to the pickup hold shelf
"""

sql = """\
with hold_shelf_data as (
  select
    date(modified_epoch, 'unixepoch', 'localtime') as date_hold_on_holdshelf,
    date(placed_epoch, 'unixepoch', 'localtime') as date_hold_placed,
    -- cast(
    --  round((modified_epoch - placed_epoch) / 86400.0) as integer
    -- ) as days_to_holdshelf,
    s_location_code as item_source_location_code,
    (
      select
        coalesce(branch_name.name, s_location_code, '')
      from
        "location"
        left outer join branch on branch.code_num = "location".branch_code_num
        left outer join branch_name on branch_name.branch_id = branch.id
      where
        "location".code = s_location_code
      limit
        1
    ) as item_source_branch_name,
    item.item_format,
    pickup_location_code,
    (
      select
        coalesce(branch_name.name, pickup_location_code, '')
      from
        "location"
        left outer join branch on branch.code_num = "location".branch_code_num
        left outer join branch_name on branch_name.branch_id = branch.id
      where
        "location".code = pickup_location_code
      limit
        1
    ) as item_pickup_branch_name
  from
    holds_shelf
    left outer join item as item on item.item_record_num = holds_shelf.record_num
  where
    modified_epoch >= CAST(
      strftime('%s', :start_date || '-01') AS INT
    )
    and modified_epoch < CAST(
      strftime('%s', DATE(:start_date || '-01', '+1 months')) AS INT
    )
)
select
  :start_date as month,
  item_source_branch_name,
  case
    when item_source_branch_name == item_pickup_branch_name then 'Items Kept For Local Branch Holdshelf'
    else 'Items Outbound to External Branch Holdshelf'
  end as hold_for_branch,
  -- item_format,
  count(*) as count
from
  hold_shelf_data
where
  item_source_branch_name is not null
group by
  1,
  2,
  3
"""

df = pd.read_csv(
        chpl_collection_url + '.csv?' + urlencode(query={'sql': sql, 'start_date': start_date})
)

alt.vconcat(
  alt.Chart(df[df['item_source_branch_name'] == 'Main Library']).mark_bar().encode(
      x=alt.X('count',),
      y=alt.Y('item_source_branch_name', sort='-x', axis=alt.Axis(title='')),
      tooltip=['month', 'item_source_branch_name', 'hold_for_branch', 'count'],
      color='hold_for_branch',
  ).properties(
      title='Item Counts for Hold-shelves - Main Library Only {}'.format(start_date)
  ),
  alt.Chart(df[df['item_source_branch_name'] != 'Main Library']).mark_bar().encode(
      x=alt.X('count',),
      y=alt.Y('item_source_branch_name', sort='-x', axis=alt.Axis(title='')),
      tooltip=['month', 'item_source_branch_name', 'hold_for_branch', 'count'],
      color='hold_for_branch',
  ).properties(
      title='Item Counts for Hold-shelves - Excluding Main Library - {}'.format(start_date)
  )
)

alt.VConcatChart(...)